# Install

In [1]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets
!pip install -q -U scipy
!pip install -q -U jupyter ipywidgets

# Import

In [5]:
import tqdm as notebook_tqdm
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, AutoModel
import transformers

# Load Quantized Model

In [6]:
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded" # sharded model by vilsonrodrigues
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [7]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Prepare Data

In [8]:
data = load_dataset("json", data_files="../data/train.json")
data

Found cached dataset json (/home/elsatch/.cache/huggingface/datasets/json/default-be29fcba34db1804/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 1/1 [00:00<00:00, 238.83it/s]


DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 252
    })
})

In [9]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

train_dataset = data['train'].shuffle().map(lambda x: {"input_text": x['prompt']  + "\n" + x['completion']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset['input_text'], truncation=True, padding=True, max_length=256, return_tensors='pt')

In [10]:
data['train'][62]

{'prompt': 'Verify if the country column consists of values USA, Canada and Mexico',
 'completion': "expect_column_values_to_be_in_set(column='country', value_set=['USA', 'Canada', 'Mexico'])"}

In [11]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [12]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

# Example Before Fine Tuning

In [13]:
def generate(index):

  example_text = data['train'][index]['prompt']
  correct_answer = data['train'][index]['completion']

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=100, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

  print("Answer:")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

  print("Best Answer:")
  print(correct_answer)

  print()

In [14]:
%%time
generate(0)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Ensure that MIT University graduate proportion of unique values is within 90% and 100%.
Answer:
Ensure that MIT University graduate proportion of unique values is within 90% and 100%.
The proportion of unique values in the MIT University graduate population is 90%.
Best Answer:
expect_column_proportion_of_unique_values_to_be_between(column='graduate', min_value=0.9, max_value=1.0, condition_parser='pandas',  row_condition='University==MIT')

CPU times: user 1.28 s, sys: 225 ms, total: 1.51 s
Wall time: 1.66 s


# Training

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=25,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        # max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

  0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_15681/2640951834.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
  1%|          | 1/100 [00:23<38:51, 23.55s/it]

{'loss': 2.4287, 'learning_rate': 4e-05, 'epoch': 0.25}


  2%|▏         | 2/100 [00:47<38:22, 23.49s/it]

{'loss': 2.4137, 'learning_rate': 8e-05, 'epoch': 0.5}


  3%|▎         | 3/100 [01:10<38:19, 23.70s/it]

...

 99%|█████████▉| 99/100 [34:19<00:19, 19.87s/it]

{'loss': 0.2775, 'learning_rate': 5.467426590739511e-08, 'epoch': 24.75}


100%|██████████| 100/100 [34:38<00:00, 20.79s/it]

{'loss': 0.2867, 'learning_rate': 0.0, 'epoch': 25.0}
{'train_runtime': 2078.9108, 'train_samples_per_second': 3.03, 'train_steps_per_second': 0.048, 'train_loss': 0.7900266298651695, 'epoch': 25.0}


TrainOutput(global_step=100, training_loss=0.7900266298651695, metrics={'train_runtime': 2078.9108, 'train_samples_per_second': 3.03, 'train_steps_per_second': 0.048, 'train_loss': 0.7900266298651695, 'epoch': 25.0})

# Example After Fine Tuning

In [21]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (dense): Linear4bit(in_

In [1]:
#from huggingface_hub import notebook_login
#notebook_login()

In [ ]:
#model.push_to_hub("DioulaD/falcon-7b-instruct-qlora-ge-dq-v2", use_auth_token=True)

In [ ]:
#tokenizer.push_to_hub("DioulaD/falcon-7b-instruct-qlora-ge-dq-v2")

In [ ]:
# model.config.push_to_hub("DioulaD/falcon-7b-instruct-qlora-ge-dq-v2")

In [23]:
pt_save_directory = "./content/falcon-7b-instruct-qlora-ge-dq-v2"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory, from_pt=True)

In [24]:
trainer.save_model(",/content/falcon-7b-instruct-qlora-ge-dq-v2")

# Inference

In [15]:
from peft import PeftModel, PeftConfig

In [17]:
# PEFT_MODEL = "DioulaD/falcon-7b-instruct-qlora-ge-dq-v2"

PEFT_MODEL = './content/falcon-7b-instruct-qlora-ge-dq-v2'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
config = PeftConfig.from_pretrained(PEFT_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)
prompt = "Verify that for rows where the category is Electronics, the userType column vaues are in the set: member, guest"

encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")

with torch.inference_mode():
    out = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        max_new_tokens=100, do_sample=True, temperature=0.3,
        eos_token_id=tokenizer.eos_token_id,
        top_k=0
    )

response = tokenizer.decode(out[0], skip_special_tokens=True)
print(response)


Loading checkpoint shards: 100%|██████████| 15/15 [00:04<00:00,  3.19it/s]
Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Verify that for rows where the category is Electronics, the userType column vaues are in the set: member, guest, and admin.
expect_column_values_to_be_in_set(column='userType', value_set=['member', 'guest', 'admin'], condition_parser='pandas', row_condition='category==Electronics')
